# python xlsx parser

In [4]:
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from itertools import islice
import datetime

date, stock = 0, 1

workbook = openpyxl.load_workbook('./Cas_pratique_-_Volumes_par_jour.xlsx')
commandes = workbook['Commandes']
date_format = "%Y-%m-%d %H:%M:%S"
prev = 1
week = 0

def get_range_from_sheet(sheet, start_row, end_row):
    # Extract the desired range of rows from the sheet
    range_of_rows = list(islice(sheet.iter_rows(values_only=True), start_row, end_row + 1))
    return range_of_rows


def create_new_file_with_range(week, week_range):
    # --------- set up ----------
    header_row = ["date", "volume", "day"] # header for sheet 1
    length = len(week_range) + 1 # number of cells
    rows = 1 
    totalVolume = 0 # to keep track of the week volumes 
    newWorkBook = openpyxl.Workbook() 
    sheet1 = newWorkBook.active 
    sheet1.title = "volume_per_days"  # Rename the first sheet
    sheet2 = newWorkBook.create_sheet(title="week_sum") # Create a new sheet
    sheet1.insert_rows(length)

    # ---------- building SXX file ---------
    for col, value in enumerate(header_row, start=1):
        sheet1.cell(row=rows, column=col, value=value)
    rows = rows + 1
    # filling new file with parsed values 
    for row in week_range:
        sheet1.cell(row=rows, column=1, value=row[date].strftime("%m/%d/%y"))
        sheet1.cell(row=rows, column=2, value=row[stock])
        sheet1.cell(row=rows, column=3, value=datetime.datetime.strptime(str(row[date]), date_format).strftime("%A"))
        totalVolume = totalVolume + int(row[stock])
        rows = rows + 1
    sheet2.insert_rows(2)
    sheet2.cell(row=1, column=1, value='total_volume') # header for sheet 2
    sheet2.cell(row=1, column=2,value='average_per_day') # header for sheet 2
    sheet2.cell(row=2, column=1, value=totalVolume)
    sheet2.cell(row=2, column=2, value=totalVolume / (length - 1))
    
    if week < 10: # i guess there is a cleaner way to do that 
        newWorkBook.save(f'S0{week}.xlsx')
    else:
        newWorkBook.save(f'S{week}.xlsx')
    newWorkBook.close()
    
    
for idx, row in enumerate(commandes.iter_rows(values_only=True), start=0):
    if not idx: # skipping header
        continue
    if (datetime.datetime.strptime(str(row[date]), date_format).strftime("%A")) == 'Monday':
        create_new_file_with_range(week, get_range_from_sheet(commandes, prev, idx - 1))
        prev = idx
        week = week + 1
    elif (idx == commandes.max_row - 1): # in case year dont end with monday
        create_new_file_with_range(week, get_range_from_sheet(commandes, prev, idx)) 


## Review

***Dependencies***

    --> The main point is the XLSX parser. For this, I used the openpyxl package. The main 
    reason for choosing it is that its usage is straightforward to understand, especially 
    when accessing cells. It is one of the first packages that came up when searching for an 
    XLSX parser, and I knew I would be able to find tutorials easily.

    --> I also used the datetime module for manipulating date formats.

    --> Additionally, I used the itertools module for convenience in certain operations.

***code***

    --> i start by loading the required file, and setting up some globals that will be use in the main loop (date format, week tracking and prev for building my week range)
    
    --> **the main loop**: I started from the assumption that every week start from monday, so, iterating trought every row of my file, each monday represent a new week. 
    i can now build a new range from the monday i just find to the previous one.
    the function create_new_file_with_range() is receiving a range and iterate trought it to file the corresponding week (file).
